In [ ]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Fluxo ótico

Considere dois quadros sucessivos de um video, e concentre-se em um ponto do espaço onde um objeto em movimento aparece. O que acontece com as imagens dos quadros de video neste ponto? Qual a relação entre $I(x,y,t)$ e $I(x,y,t + \Delta t)$?

Vamos analisar os quadros do video a seguir:

In [ ]:
import os.path

cap = cv2.VideoCapture('slow_traffic_small.mp4')

_, frame = cap.read()

while frame is not None:
    cv2.imshow('frame', frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    _, frame = cap.read()

cap.release()
cv2.destroyAllWindows()

Vamos nos concentrar nas regiões delimitadas abaixo, nos quadros $100$ e $101$. A região $1$ apresenta movimento, enquanto a região $2$ é um segmento estático do video.

In [ ]:
from matplotlib.patches import Rectangle

def add_box(reg, color, label):
    x, y, width, height = reg
    plt.gca().add_patch(Rectangle((x, y), width, height, linewidth=1, edgecolor=color, facecolor='none'))
    plt.text(x, y - 5, label, color=color)

img_antes = cv2.imread('./Imagens/frame_0100.png')
img_depois = cv2.imread('./Imagens/frame_0101.png')

reg1 = (400, 250, 50, 50)
reg2 = (400, 75, 50, 50)

plt.figure(figsize=(16, 16))

plt.subplot(2, 1, 1)
plt.imshow(img_antes[:, :, ::-1])
add_box(reg1, 'red', 'Região 1')
add_box(reg2, 'blue', 'Região 2')
plt.title('Antes')

plt.subplot(2, 1, 2)
plt.imshow(img_depois[:, :, ::-1])
add_box(reg1, 'red', 'Região 1')
add_box(reg2, 'blue', 'Região 2')
plt.title('Depois')

plt.show()

Vamos observar as regiões em detalhe:

In [ ]:
def corta_reg(img, reg):
    x, y, width, height = reg
    if len(img.shape) == 2:
        return img[y:(y+height), x:(x+width)]
    else:
        return img[y:(y+height), x:(x+width), :]

def show_reg(img, reg):
    if len(img.shape) == 2:
        plt.imshow(corta_reg(img, reg)[:, :], cmap='gray')
    else:
        plt.imshow(corta_reg(img, reg)[:, :, ::-1])

plt.figure(figsize=(16,16)) 

plt.subplot(2, 2, 1)
show_reg(img_antes, reg1)
plt.title('Antes, região 1')

plt.subplot(2, 2, 2)
show_reg(img_depois, reg1)
plt.title('Depois, região 1')

plt.subplot(2, 2, 3)
show_reg(img_antes, reg2)
plt.title('Antes, região 2')

plt.subplot(2, 2, 4)
show_reg(img_depois, reg2)
plt.title('Depois, região 2')

plt.show()

Podemos observar que houve alguma mudança na região $1$, onde temos um carro em movimento, mas não na região $2$. 

Vamos trabalhar apenas com a imagem em tons de cinza, meramente para simplificar nossa explicação matemática a seguir:

In [ ]:
img_antes_gray = cv2.cvtColor(img_antes, cv2.COLOR_BGR2GRAY) / 255.0
img_depois_gray = cv2.cvtColor(img_depois, cv2.COLOR_BGR2GRAY) / 255.0

Note a diferença entre quadros ponto-a-ponto:

In [ ]:
img_diff_reg_1 = corta_reg(img_depois_gray, reg1) - corta_reg(img_antes_gray, reg1)
img_diff_reg_2 = corta_reg(img_depois_gray, reg2) - corta_reg(img_antes_gray, reg2)

plt.figure(figsize=(16,8))

plt.subplot(1, 2, 1)
plt.imshow(img_diff_reg_1, vmin = -1.0, vmax = 1.0, cmap='gray')
plt.title(f'Região 1, diff em [{np.min(img_diff_reg_1):.2f}, {np.max(img_diff_reg_1):.2f}]')

plt.subplot(1, 2, 2)
plt.imshow(img_diff_reg_2, vmin = -1.0, vmax = 1.0, cmap='gray')
plt.title(f'Região 2, diff em [{np.min(img_diff_reg_2):.2f}, {np.max(img_diff_reg_2):.2f}]')

plt.show()

Observe que houve bastante mudança na região $1$ e pouca mudança na região $2$. 

Prestando atenção no detalhe do detalhe vermelho do parachoque, podemos estimar que o movimento vertical da imagem do carro foi para baixo e para a direita. Vamos comparar este detalhe entre os sucesssivos quadros, mas incorporando a translação no quadro posterior. O valor da translação é mágico, por enquanto...

In [ ]:
M = np.array([
    [1.0, 0.0, -0.58],  # 1.8 para baixo.
    [0.0, 1.0, -0.23],  # 0.7 para a direita.
])

img_depois_gray_transl = cv2.warpAffine(img_depois_gray, M, None)
img_diff_transl = corta_reg(img_depois_gray_transl, reg1) - corta_reg(img_depois_gray, reg1)

plt.figure(figsize=(18,6))

plt.subplot(1, 3, 1)
show_reg(img_antes_gray, reg1)
plt.title('Região 1')

plt.subplot(1, 3, 2)
show_reg(img_depois_gray_transl, reg1)
plt.title('Região 1 transladada')

plt.subplot(1, 3, 3)
plt.imshow(img_diff_transl, vmin = -1.0, vmax = 1.0, cmap='gray')
plt.title(f'Região 2, diff em [{np.min(img_diff_transl):.2f}, {np.max(img_diff_transl):.2f}]')

plt.show()

Note que a diferença entre quadros diminuiu ao se incorporar a translação, como esperado.

Desta observação vem a idéia central da técnica de *fluxo ótico*: determinar para cada ponto da imagem uma translação tal que:

$$
I(x, y, t) = I(x + \Delta x, y + \Delta y, t + \Delta t)
$$

onde $\Delta t$ é o espaçamento temporal entre quadros, $\Delta x$ e $\Delta y$ são o deslocamento do objeto entre quadros. Esta é a hipótese do fluxo ótico.

Nosso objetivo é determinal $\Delta x$ e $\Delta y$. Decompondo $I(x + \Delta x, y + \Delta y, t + \Delta t)$ em série de Taylor temos:

$$
I(x + \Delta x, y + \Delta y, t + \Delta t) \approx I(x, y, t) 
+ \frac{\partial I}{\partial x}(x, y, t) \Delta x 
+ \frac{\partial I}{\partial y}(x, y, t) \Delta y
+ \frac{\partial I}{\partial t}(x, y, t) \Delta t 
$$

Substituindo a hipótese do fluxo ótico temos:

$$
\frac{\partial I}{\partial x}(x, y, t) \Delta x 
+ \frac{\partial I}{\partial y}(x, y, t) \Delta y
+ \frac{\partial I}{\partial t}(x, y, t) \Delta t = 0
$$

Dividindo por $\Delta t$ obtemos:

$$
\frac{\partial I}{\partial x}(x, y, t) v_x(x, y, t)
+ \frac{\partial I}{\partial y}(x, y, t) v_y(x, y, t) + \frac{\partial I}{\partial t}(x, y, t) = 0
$$

Para maior clareza, vamos remover as indicações de coordenadas $(x, y, t)$, e abreviar as derivadas parciais em $x$, $y$ e $t$ por $I_x$, $I_y$ e $I_t$ respectivamente. A equação acima fica sendo:

$$
I_x v_x + I_y v_y + I_t = 0
$$

Temos apenas uma equação e duas incógnitas. Como resolver esse problema?

In [ ]:
img = img_antes_gray

kd, k = cv2.getDerivKernels(1, 0, 1, normalize=True)

print('Kernel para derivada')
print(kd)

print('Kernel sem derivada')
print(k)

Ix = cv2.sepFilter2D(img, -1, kd, k)
Iy = cv2.sepFilter2D(img, -1, k, kd)
It= img_depois_gray - img_antes_gray

plt.figure(figsize=(16, 9))

plt.subplot(2, 2, 1)
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title('img')

plt.subplot(2, 2, 2)
plt.imshow(Ix, cmap='gray')
plt.axis('off')
plt.title('$I_x$')

plt.subplot(2, 2, 3)
plt.imshow(Iy, cmap='gray')
plt.axis('off')
plt.title('$I_y$')

plt.subplot(2, 2, 4)
plt.imshow(It, cmap='gray')
plt.axis('off')
plt.title('$I_t$')

plt.show()

## Algoritmo de Lucas-Kanade

A idéia do algoritmo de Lucas-Kanade é supor que o movimento é mais ou menos constante em uma vizinhança em torno de um pixel dado. Isso vai falhar nas bordas de objetos, mas paciência. Dentro dessa hipótese, vamos considerar uma vizinhança ${x_i, y_i}$ em torno de $(x, y)$, e escrever a equação do fluxo ótico para cada um desses pontos, mas mantendo o mesmo $v_x$ e $v_y$:

$$
\begin{split}
& I_x(x_1, y_1, t) v_x + I_y(x_1, y_1, t) v_y + I_t(x_1, y_1, t) = 0 \\
& I_x(x_2, y_2, t) v_x + I_y(x_2, y_2, t) v_y + I_t(x_2, y_2, t) = 0 \\
& \vdots \\
& I_x(x_n, y_n, t) v_x + I_y(x_n, y_n, t) v_y + I_t(x_n, y_n, t) = 0 
\end{split}
$$

Agora temos o problema oposto: muitas equações para apenas duas incógnitas!

Vamos modificar um pouco o sistema de equações e introduzir um termo de "erro" em cada equação. Esta é a abordagem padrão do *método dos mínimos quadrados*:

$$
\begin{split}
& I_x(x_1, y_1, t) v_x + I_y(x_1, y_1, t) v_y + I_t(x_1, y_1, t) = \varepsilon_1 \\
& I_x(x_2, y_2, t) v_x + I_y(x_2, y_2, t) v_y + I_t(x_2, y_2, t) = \varepsilon_2 \\
& \vdots \\
& I_x(x_n, y_n, t) v_x + I_y(x_n, y_n, t) v_y + I_t(x_n, y_n, t) = \varepsilon_n 
\end{split}
$$

Em notação matricial, temos:

$$
\underbrace{
\left[
\begin{matrix}
I_x(x_1, y_1, t) & I_y(x_1, y_1, t)\\
I_x(x_2, y_2, t) & I_y(x_2, y_2, t)\\
\vdots & \vdots\\
I_x(x_n, y_n, t) & I_y(x_n, y_n, t) 
\end{matrix}
\right]
}_{\mathbf{A}} 
\underbrace{
\left[
\begin{matrix}
v_x \\
v_y
\end{matrix}
\right]
}_{\mathbf{v}}
-
\underbrace{
\left[
\begin{matrix}
-I_t(x_1, y_1, t)\\
-I_t(x_2, y_2, t)\\
\vdots\\
-I_t(x_n, y_n, t)
\end{matrix}
\right]
}_{\mathbf{b}}
=
\underbrace{
\left[
\begin{matrix}
\varepsilon_1\\
\varepsilon_2\\
\vdots\\
\varepsilon_n
\end{matrix}
\right]
}_{\mathbf{\varepsilon}}
$$

ou 

$$
\mathbf{A} \mathbf{v} - \mathbf{b} = \mathbf{\varepsilon}
$$

A soma dos quadrados das componentes de erro é:

$$
S = \sum_{i = 1}^{n} \varepsilon_i^2 = \mathbf{\varepsilon}^T \mathbf{\varepsilon}
$$

Queremos encontrar um valor de $\mathbf{v}$ que minimize $S$. Substituindo a equação anterior temos:

$$
S = (\mathbf{A} \mathbf{v} - \mathbf{b})^T (\mathbf{A} \mathbf{v} - \mathbf{b}) = 
\mathbf{v}^T \mathbf{A}^T \mathbf{A} \mathbf{v} 
    - 2 \mathbf{v}^T \mathbf{A}^T \mathbf{b}
    + \mathbf{b}^T \mathbf{b}
$$

Derivando em relação a $\mathbf{v}$ (veja https://en.wikipedia.org/wiki/Matrix_calculus) temos o seguinte:

$$
\begin{split}
\frac{\partial}{\partial \mathbf{v}} \left(\mathbf{v}^T \mathbf{A}^T \mathbf{A} \mathbf{v}\right) & = 2 \mathbf{A}^T \mathbf{A} \mathbf{v}\\
\frac{\partial}{\partial \mathbf{v}} \left(\mathbf{v}^T \mathbf{A}^T \mathbf{b}\right) & = \mathbf{A}^T \mathbf{b} \\
\frac{\partial}{\partial \mathbf{v}} \left(\mathbf{b}^T \mathbf{b}\right) & = 0 \\
\end{split}
$$

Consequentemente, tomando a derivada de $S$ em relação a $\mathbf{v}$ temos:

$$
\frac{\partial S}{\partial \mathbf{v}} = 2 \mathbf{A}^T \mathbf{A} \mathbf{v} - 2 \mathbf{A}^T \mathbf{b}
$$

Igualando esta expressão a zero ***e supondo que $\mathbf{A}^T \mathbf{A}$ é inversível*** obtemos a solução do nosso problema:

$$
2 \mathbf{A}^T \mathbf{A} \mathbf{v} - 2 \mathbf{A}^T \mathbf{b} = 0 \Longrightarrow \mathbf{v} = (\mathbf{A}^T \mathbf{A})^{-1} \mathbf{A}^T \mathbf{b}
$$

Substituindo as definições dos vários termos, temos a forma final da expressão de calculo do fluxo otico em um ponto $(x, y, t)$:

$$
\left[
\begin{matrix}
v_x \\
v_y
\end{matrix}
\right]
=
\left[
\begin{matrix}
\sum I_x^2 & \sum I_x I_y\\
\sum I_x I_y & \sum I_y ^2\\
\end{matrix}
\right]^{-1}
\left[
\begin{matrix}
-\sum I_x I_t\\
-\sum I_y I_t\\
\end{matrix}
\right]
$$

**Atividade**: Calcule o deslocamento ótimo para o nosso caso de exemplo usando o resultado acima.

Dica: $\sum I_x^2$ é simplesmente `np.sum(Ix*Ix)`, etc..

É razoavelmente simples calcular o deslocamento ótimo para todos os pontos da imagem usando numpy:

- Como a matriz $\mathbf{A}$ é de tamanho $2 \times 2$, podemos determinar sua inversa algebricamente sem maiores problemas. 
- Os somatórios locais podem ser calculados com convolução.
- As operações pixel-a-pixel podem ser feitas como somas e multiplicações de arrays no numpy

Trata-se de uma implementação "caseira" de fluxo ótico, mas já dá para usar para aprender sobre o método.

Antes de prosseguir, temos um problema: o que acontece se a matriz $\mathbf{A}$ não for "bem inversível"? Por enquanto vamos pular esse problema com uma gambiarra numérica - veja no código abaixo.

In [ ]:
a = cv2.GaussianBlur(Ix*Ix, (15, 15), 3)
b = cv2.GaussianBlur(Ix*Iy, (15, 15), 3)
c = cv2.GaussianBlur(Iy*Iy, (15, 15), 3)
d = -cv2.GaussianBlur(Ix*It, (15, 15), 3)
e = -cv2.GaussianBlur(Iy*It, (15, 15), 3)

det = a*c - b*b
inv_det = np.sign(det) * (1.0 / (np.abs(det) + 1e-4))

vx = (c*d - b*e) * inv_det
vy = (a*e - b*d) * inv_det

vmag = np.sqrt(vx**2 + vy**2)
vang = np.arctan2(vy, vx)


vmask = vmag < np.percentile(vmag, 90)
vang[vmask] = 0.0

plt.figure(figsize=(16, 15))

plt.subplot(3, 2, 1)
plt.imshow(img_antes[:,:,::-1], cmap='gray')
plt.title('Antes')
plt.axis('off')

plt.subplot(3, 2, 2)
plt.imshow(img_depois[:,:,::-1], cmap='gray')
plt.title('Depois')
plt.axis('off')

plt.subplot(3, 2, 3)
plt.imshow(vx, cmap='gray')
plt.title(f'{np.min(vx):.2f} $\leq v_x \leq$ {np.max(vx):.2f} ')
plt.axis('off')

plt.subplot(3, 2, 4)
plt.imshow(vy, cmap='gray')
plt.title(f'{np.min(vy):.2f} $\leq v_y \leq$ {np.max(vy):.2f} ')
plt.axis('off')

plt.subplot(3, 2, 5)
plt.imshow(vmag, cmap='gray')
plt.title(f'{np.min(vmag):.2f} $\leq |\mathbf{{v}}| \leq$ {np.max(vmag):.2f} ')
plt.axis('off')

plt.subplot(3, 2, 6)
plt.imshow(vang, cmap='gray')
plt.title(f'{np.min(vang):.2f} $\leq ang(\mathbf{{v}}) \leq$ {np.max(vang):.2f} ')
plt.axis('off')

plt.show()

Observe que os pontos onde o deslocamento for melhor calculado são as regiões não-planas, preferencialmente as quinas ou cruzamentos...

## O problema do condicionamento de $\mathbf{A}$: *Good Features to Track*

É fundamental que a matriz $\mathbf{A}$ seja "bem inversível", caso contrário não teremos boa acurácia numérica no método, ou mesmo não teremos nem ao menos a derivada!

O que quer dizer "bem inversível"? Para que não tenhamos uma explosão numérica no processo de inversão de matriz, é necessário que ambos os autovalores de $\mathbf{A}$ tenham magnitude bem maior que zero, e que tais magnitudes sejam similares entre si.

Oras, essas são exatamente as condições do detector de Shi-Tomasi, o "Good Features To Track"! Agora ficou claro o porquê do nome deste detector de características visuais. 

## Como detectar grandes deslocamentos?

Todo o algoritmo está baseado na expansão em série de Taylor para um deslocamento pequeno. Como fazer para detectar um deslocamento grande?

Um deslocamento grande pode ser visto como um deslocamento pequeno, mas em uma escala maior! Uma ideia portanto seria:

1. Decompor os quadros em pirâmide.
2. Executar o Shi-Tomasi para descobrir pontos notáveis na escala maior em ambos os quadros.
3. Efetuar o Lucas-Kanade para determinar o fluxo ótico em torno de cada ponto notável, e fazer a correspondência entre pontos notáveis dos quadros anterior e posterior (rastreamento intra-escala).
4. Subir um nível na pirâmide, refazer o Shi-Tomasi e fazer a correspondência entre pontos notáveis de escalas diferentes (rastreamento inter-escala)
5. Repetir passo 4 até o fim das escalas.

Esta é a implementação do Lucas-Kanade piramidal, descrita no artigo de Bouguet (ver no Blackboard), e está implementada no método `cv2.calcOpticalFlowPyrLK()`. Veja abaixo um exemplo de rastreamento de movimento usando essa técnica.

In [ ]:
# Adaptado do exemplo do OpenCV em https://docs.opencv.org/ref/master/d4/dee/tutorial_optical_flow.html
cap = cv2.VideoCapture('slow_traffic_small.mp4')

# Troque a linha anterior pela linha seguinte para usar a cãmera do laptop.
#cap = cv2.VideoCapture(0)


# Parameters for Shi-Tomasi corner detection.
feature_params = {
    'maxCorners': 100,
    'qualityLevel': 0.3,
    'minDistance': 7,
    'blockSize': 7
}

# Parameters for Lucas-Kanade optical flow.
lk_params = {
    'winSize': (15, 15),
    'maxLevel': 2,
    'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
}

# Create some random colors.
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it.
_, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes.
mask = np.zeros_like(old_frame)

while True:
    _, frame = cap.read()
    if frame is None:
        break
        
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow.
    p1, st, err = cv2.calcOpticalFlowPyrLK(
        old_gray, frame_gray, p0, None, **lk_params)

    # Select good points.
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks.
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)

    cv2.imshow('frame', img)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points.
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()    
cv2.destroyAllWindows()

## Projeto 2-1: Estabilização de imagens

**Motivação**: https://www.youtube.com/watch?v=4vt7bGEen2s

Agora que você consegue capturar o fluxo ótico entre duas imagens consecultivas, vamos utilizá-lo de modo inverso: como seria uma forma de compensar eventuais oscilações na câmera?

Projete um programa que captura as imagens da webcam e realiza a estabilização da imagem. Você notará que se utilizar o programa acima, a estabilização será parcial e falha. Por que?

Você deve construir um Jupyter Notebook que utiliza o fluxo ótico para corrigir o problema acima. O notebook deve conter comentários acerca da solução usada. Você pode usar o algoritmo Lucas-Kanade piramidal, explicado neste notebook, ou o algoritmo de fluxo otico denso (algoritmo de Farneback) implementado na função `cv2.calcOpticalFlowFarneback()` (que vai provavelmente ser muito mais simples para videos pequenos!)

O programa base para uso da webcam:

```Python
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)
 
while True:
    _, frame = captura.read()
    if frame is None:
        break
        
    cv2.imshow("Video", frame)
   
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
 
captura.release()
cv2.destroyAllWindows()
```

### Entrega: 25/Set 23:59 via GitHub.

### Rubrica do Projeto:

    I. Não entregou ou entregou apenas um rascunho.
    D. O programa faz apenas uma estabilização parcial com os programas da aula de hoje.
    C. Utiliza o fluxo otico de uma janela no centro da imagem.
    B. Utiliza meios para compensar as faixas pretas nos cantos da imagem com algum limite.
    A. Consegue realizar a compensação em rotação no eixo de profundidade.
    
    +1/2 Conceito para implementações que comprovadamente melhoram o desempenho da estabilização.
    -1/2 Conceito se o notebook não contiver uma explicação detalhada da solução apresentada.

In [3]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [4]:
# Adaptado do exemplo do OpenCV em https://docs.opencv.org/ref/master/d4/dee/tutorial_optical_flow.html
#cap = cv2.VideoCapture('slow_traffic_small.mp4')

# Troque a linha anterior pela linha seguinte para usar a cãmera do laptop.
cap = cv2.VideoCapture(0)

# Parameters for Shi-Tomasi corner detection.
feature_params = {
    'maxCorners': 100,
    'qualityLevel': 0.3,
    'minDistance': 7,
    'blockSize': 7
}

# Parameters for Lucas-Kanade optical flow.
lk_params = {
    'winSize': (15, 15),
    'maxLevel': 2,
    'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
}

# Create some random colors.
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it.
_, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes.
mask = np.zeros_like(old_frame)

movx=0
movy=0

while True:
    _, frame = cap.read()
    if frame is None:
        break
        
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame_gray = frame_gray[300:300][300:300][:]

    # Calculate optical flow.
    p1, st, err = cv2.calcOpticalFlowPyrLK(
        old_gray, frame_gray, p0, None, **lk_params)
    
    vecx = []
    vecy = []
    
    try: 
        for i,(point) in enumerate(p1):
            vec_p = (point[0]-(p0[i][0]))

            vecx.append((vec_p[0]))
            vecy.append((vec_p[1]))
    except:
        break
        
        
    movx += np.average(vecx)
    movy += np.average(vecy)
    
    M = np.array([[1,0,-movx],[0,1,-movy]])
    rows,cols,channel = frame.shape
    
    #frame=cv2.warpAffine(frame,M,(cols,rows))
        
    # Select good points.
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    '''
    # Draw the tracks.
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
    '''
    frame = cv2.add(frame, mask)
    img=cv2.warpAffine(frame,M,(cols,rows))
    
    
    
    
    cv2.imshow('frame', img)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points.
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()    
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
np.ceil(rows-movx)